In [15]:
import numpy as np
import pandas as pd
import shutil
from sklearn.ensemble import BaggingClassifier
import os
from os import listdir
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
from natsort import natsorted
import cv2
import os
import h5py


import warnings
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals import joblib

In [33]:
labels = pd.read_csv("./bonus-sml-2020/SML_Train.csv")

# Create `train_sep` directory
train_dir = './bonus-sml-2020/SML_Train/'
train_sep_dir = './bonus-sml-2020/train_sep/'
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)

for filename, class_name in labels.values:
    # Create subdirectory with `class_name`
    if not os.path.exists(train_sep_dir + str(class_name)):
        os.mkdir(train_sep_dir + str(class_name))
    src_path = train_dir + filename
    dst_path = train_sep_dir + str(class_name) + '/' + filename
    try:
        shutil.copy(src_path, dst_path)
    except IOError as e:
        print('Unable to copy file {} to {}'
              .format(src_path, dst_path))
    except:
        print('When try copy file {} to {}, unexpected error: {}'
              .format(src_path, dst_path, sys.exc_info()))

In [34]:
#images_per_class = 80
fixed_size       = tuple((500, 500))
train_path       = "./bonus-sml-2020/train_sep/"
h5_data          = './bonus-sml-2020/output/data.h5'
h5_labels        = './bonus-sml-2020/output/labels.h5'
bins             = 8

In [35]:
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [36]:
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

In [37]:
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [38]:
train_labels = os.listdir(train_path)
print(train_labels)
train_labels = list(map(int, train_labels))

# sort the training labels
train_labels.sort()
print(train_labels)

# empty lists to hold feature vectors and labels
global_features = []
labels          = []

['20', '18', '9', '0', '11', '7', '16', '6', '17', '1', '10', '19', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [39]:
# mom: (7,)
# har: (13,)
# hist: (512,)
# gloval: (532,)

In [40]:
for training_name in train_labels:
    # join the training data path and each species training folder
    dir = os.path.join(train_path, str(training_name))

    # get the current training label
    current_label = training_name
    
    # loop over the images in each sub-folder
    for file in listdir(dir):
        # get the image file name
        # read the image and resize it to a fixed-size
        image = cv2.imread(dir+'/'+file)
        image = cv2.resize(image, fixed_size)

        ####################################
        # Global Feature extraction
        ####################################
        fv_hu_moments = fd_hu_moments(image)
        #print("mom:",fv_hu_moments.shape)
        fv_haralick   = fd_haralick(image)
        #print("har:",fv_haralick.shape)
        fv_histogram  = fd_histogram(image)
        #print("hist:",fv_histogram.shape)
        ###################################
        # Concatenate global features
        ###################################
        global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
        #print("gloval:",global_feature.shape)

        # update the list of labels and feature vectors
        labels.append(current_label)
        global_features.append(global_feature)

    print(len(global_features[0]))

print("Done")

532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
Done


In [41]:
print(np.array(global_features).shape)

# get the overall training label size
print(np.array(labels).shape)

# encode the target labels
# targetNames = np.unique(labels)
# le          = LabelEncoder()
# target      = le.fit_transform(labels)
# print("[STATUS] training labels encoded...")
target=labels
print(np.array(target).shape)

# scale features in the range (0-1)
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)


# save the feature vector using HDF5
h5f_data = h5py.File(h5_data, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File(h5_labels, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")

(16000, 532)
(16000,)
(16000,)
[STATUS] end of training..


In [42]:
num_trees = 700
test_size = 0.10
seed      = 9
train_path = "./bonus-sml-2020/train_sep/"
test_path  = "./bonus-sml-2020/SML_Test/"
h5_data          = './bonus-sml-2020/output/data.h5'
h5_labels        = './bonus-sml-2020/output/labels.h5'
scoring    = "accuracy"

In [43]:
models = []
#models.append(('LR', LogisticRegression(random_state=seed)))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier(random_state=seed)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed,min_samples_split=2)))
print(models)
#models.append(('GB',GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)))
#models.append(('NB', GaussianNB()))
#svm=SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005,random_state=seed)
#models.append(('SVM', SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005,random_state=seed) ))
#models.append(('SVM', BaggingClassifier(base_estimator=svm, n_estimators=100, random_state=42) ))
#models.append((BaggingClassifier(base_estimator=svm, n_estimators=31, random_state=42))
#models.append(('SVM', SVC(random_state=seed)))


results = []
names   = []

# import the feature vector and trained labels
h5f_data  = h5py.File(h5_data, 'r')
h5f_label = h5py.File(h5_labels, 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string   = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels   = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

print(np.array(global_features).shape)

# get the overall training label size
print(np.array(global_labels).shape)

[('RF', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False))]
(16000, 532)
(16000,)


In [44]:
# split the training and testing data
(trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal) = train_test_split(np.array(global_features),
                                                                                          np.array(global_labels),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)

print("[STATUS] splitted train and test data...")
print("Train data  : {}",trainDataGlobal.shape)
print("Test data   : {}",testDataGlobal.shape)
print("Train labels: {}",trainLabelsGlobal.shape)
print("Test labels : {}",testLabelsGlobal.shape)

[STATUS] splitted train and test data...
Train data  : {} (14400, 532)
Test data   : {} (1600, 532)
Train labels: {} (14400,)
Test labels : {} (1600,)


In [ ]:
for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model, trainDataGlobal, trainLabelsGlobal, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Machine Learning algorithm comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [45]:
global_features_test=[]
for file in natsorted(listdir(test_path)):
        # get the image file name
        # read the image and resize it to a fixed-size
        image = cv2.imread(test_path+file)
        image = cv2.resize(image, fixed_size)

        ####################################
        # Global Feature extraction
        ####################################
        fv_hu_moments = fd_hu_moments(image)
        #print("mom:",fv_hu_moments.shape)
        fv_haralick   = fd_haralick(image)
        #print("har:",fv_haralick.shape)
        fv_histogram  = fd_histogram(image)
        #print("hist:",fv_histogram.shape)
        ###################################
        # Concatenate global features
        ###################################
        global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
        #print("gloval:",global_feature.shape)

        # update the list of labels and feature vectors
        global_features_test.append(global_feature)

print("Done")


Done


In [46]:
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features_test = scaler.fit_transform(global_features_test)


In [47]:
print(np.array(rescaled_features_test).shape)

(1500, 532)


In [48]:
import matplotlib.pyplot as plt

# create the model - Random Forests
clf  = RandomForestClassifier(n_estimators=num_trees, random_state=seed)
#clf=LogisticRegression(random_state=seed)
#clf= SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005)
# fit the training data to the model
clf.fit(trainDataGlobal, trainLabelsGlobal)
prediction=[]

y_test = clf.predict(rescaled_features_test)

In [49]:
print(y_test[0:20])

[ 3  1  6 16 14  9 17 22 24 16 24 10  3  9 17 22  4 17 19 17]


In [50]:
print(len(y_test))

1500


In [51]:
def helper(direc):
    files=[]
    for filename in natsorted(listdir(direc)):
        files.append(filename)
    return files

files=helper('./bonus-sml-2020/SML_Test/')
data_res=pd.DataFrame()
data_res['id']=files
data_res['category']=y_test
data_res.to_csv (r'sample_submission.csv', index = False, header=True)

# next


In [ ]:
for training_name in train_labels:
    # join the training data path and each species training folder
    dir = os.path.join(train_path, str(training_name))
    # loop over the images in each sub-folder
    count=
    for file in listdir(dir):
        
        # get the image file name
        # read the image and resize it to a fixed-size
        image = cv2.imread(dir+'/'+file)
        image = cv2.resize(image, fixed_size)

        ####################################